In [18]:
import pandas as pd
import datetime
import numpy as np
data = pd.read_csv('./handel/work2.csv')

In [19]:
X = data['发动机转速']
V = data['GPS车速']
A = data['加速度']

In [20]:
# 转数小于800且速度为0的静止点：怠速点
dai_points = data[(V==0) & (X<=800)]

In [21]:
# 怠速点 下标
dai_points_index = data[(V==0) & (X<=800)].index

In [22]:
# 算法 进行怠速区间划分

# 整体区间列表，元素为tuple(consistant，left, right)
# 若是连续区间,则consistant为True
idling_list = []
j = 1
n = 0 
for i in range(len(dai_points_index)-1):
    delt = dai_points_index[j] - dai_points_index[i]
    if delt>1.0:
        # 保存连续怠速区间下标
        idling_list.append((True, dai_points_index[i-n], dai_points_index[i]))
        
        # 保存不连续的怠速区间下标
        idling_list.append((False, dai_points_index[i], dai_points_index[j]))
        
        n = 0
    else:
        n = n + 1 
    j = j + 1  

In [23]:
idling_list

[(True, 0, 87),
 (False, 87, 96),
 (True, 96, 191),
 (False, 191, 658),
 (True, 658, 760),
 (False, 760, 762),
 (True, 762, 763),
 (False, 763, 820),
 (True, 820, 821),
 (False, 821, 828),
 (True, 828, 829),
 (False, 829, 853),
 (True, 853, 854),
 (False, 854, 1145),
 (True, 1145, 1200),
 (False, 1200, 1374),
 (True, 1374, 1381),
 (False, 1381, 1407),
 (True, 1407, 1560),
 (False, 1560, 1568),
 (True, 1568, 1571),
 (False, 1571, 1671),
 (True, 1671, 1789),
 (False, 1789, 1791),
 (True, 1791, 1810),
 (False, 1810, 2242),
 (True, 2242, 2253),
 (False, 2253, 3390),
 (True, 3390, 3413),
 (False, 3413, 3432),
 (True, 3432, 3439),
 (False, 3439, 3442),
 (True, 3442, 3444),
 (False, 3444, 4254),
 (True, 4254, 4254),
 (False, 4254, 4256),
 (True, 4256, 4256),
 (False, 4256, 4270),
 (True, 4270, 4270),
 (False, 4270, 4272),
 (True, 4272, 4272),
 (False, 4272, 4274),
 (True, 4274, 4274),
 (False, 4274, 4522),
 (True, 4522, 4527),
 (False, 4527, 4530),
 (True, 4530, 4533),
 (False, 4533, 4538),
 

In [24]:
# 删除所有长度大于180s的怠速区间
def delete_three(idlings):
    res = []
    for tu in idlings:
        if (tu[2] - tu[1]) <= 180:
            res.append(tu)
    return res

In [25]:
# 判断不连续怠速区间 是否有速度超过10km/h的点,若有则False
def check_idling(tu):
    # 不是怠速区间
    if np.max(V[tu[1]:tu[2]+1]) > 10.0:
        return False
    else:
        return True

In [26]:
idling_scale = delete_three(idling_list)
result_list = []
for tu in idling_scale:
    # 连续怠速区间 直接加入
    if tu[0]:
        result_list.append(tu)
    # 对不连续的怠速区间进行判断
    else:
        if check_idling(tu):
            result_list.append(tu)

In [27]:
# 最终怠速区间
result_list

[(True, 0, 87),
 (False, 87, 96),
 (True, 96, 191),
 (True, 658, 760),
 (False, 760, 762),
 (True, 762, 763),
 (True, 820, 821),
 (False, 821, 828),
 (True, 828, 829),
 (True, 853, 854),
 (True, 1145, 1200),
 (True, 1374, 1381),
 (False, 1381, 1407),
 (True, 1407, 1560),
 (False, 1560, 1568),
 (True, 1568, 1571),
 (True, 1671, 1789),
 (False, 1789, 1791),
 (True, 1791, 1810),
 (True, 2242, 2253),
 (True, 3390, 3413),
 (True, 3432, 3439),
 (False, 3439, 3442),
 (True, 3442, 3444),
 (True, 4254, 4254),
 (False, 4254, 4256),
 (True, 4256, 4256),
 (False, 4256, 4270),
 (True, 4270, 4270),
 (True, 4272, 4272),
 (False, 4272, 4274),
 (True, 4274, 4274),
 (True, 4522, 4527),
 (False, 4527, 4530),
 (True, 4530, 4533),
 (False, 4533, 4538),
 (True, 4538, 4541),
 (True, 4587, 4635),
 (True, 4650, 4738),
 (False, 4738, 4746),
 (True, 4746, 4763),
 (False, 4763, 4771),
 (True, 4771, 4826),
 (True, 5269, 5270),
 (True, 5460, 5463),
 (True, 5811, 5834),
 (False, 5834, 5845),
 (True, 5845, 5845),
 (F

In [28]:
# 原始怠速区间
len(idling_list)

4026

In [29]:
# 去掉>180
len(idling_scale)

3828

In [30]:
# 去掉<10km/h，最终怠速区间
len(result_list)

3217

In [31]:
a_v_list = []
a_s_list = []
a_plus_list = []
a_mis_list = []
idling_rate_list = []
a_pos_list = []
a_neg_list = []
v_std_list = []
a_std_list = []

for i in range(len(result_list)-1):
    end_tu = result_list[i+1]
    start_tu = result_list[i]
    
    # 怠速区间[idling_start,idling_end]
    idling_start = start_tu[1]
    idling_end = start_tu[2]
    
    # 运动学片段区间[idling_start,idling_end]
    fragment_start = idling_start
    fragment_end = end_tu[1]
    
    # 全区间时间点长度
    all_length = fragment_end - fragment_start + 1
    
    # 怠速区间时间点个数
    idling_length = idling_end - idling_start + 1
    
    # 平均速度
    a_v = np.mean(V[fragment_start:fragment_end+1])
    a_v_list.append(a_v)
    # 平均行驶速度
    a_s = np.mean(V[idling_end:fragment_end+1])
    a_s_list.append(a_s)
    
    
    # 全区间加速度
    a_list = A[fragment_start:fragment_end+1]
    # 全区间加速度
    a_jia = a_list[a_list>=0]
    # 全区间减速度
    a_jian = a_list[a_list<0]
    
    # 平均加速度
    a_plus = np.sum(a_jia) / all_length
    a_plus_list.append(a_plus)
    
    # 平均减速度
    a_mis = np.sum(a_jian) / all_length
    a_mis_list.append(a_mis)
    
    # 怠速时间比
    idling_rate = idling_length / all_length
    idling_rate_list.append(idling_rate)
    
    # 加速时间比
    a_pos = len(a_jia) / all_length
    a_pos_list.append(a_pos)
    
    # 减速时间比
    a_neg = len(a_jian) / all_length
    a_neg_list.append(a_neg)
    
    # 速度标准差
    v_std = np.std(V[fragment_start:fragment_end+1])
    v_std_list.append(v_std)
    
    # 加速度标准差
    a_std = np.std(a_jia)
    a_std_list.append(a_std)

In [32]:
prepare_pca = pd.DataFrame({
    '平均速度': a_v_list,
    '平均行驶速度': a_s_list,
    '平均加速度': a_plus_list,
    '平均减速度': a_mis_list,
    '怠速时间比': idling_rate_list,
    '加速时间比': a_pos_list,
    '减速时间比': a_neg_list,
    '速度标准差': v_std_list,
    '加速度标准差': a_std_list
})

In [33]:
prepare_pca.to_csv('prepare_pca2.csv', index=0)